In [4]:
import pandas as pd
import numpy as np
import copy
from scipy import stats
from sklearn.utils.random import check_random_state
from sklearn.ensemble import RandomForestRegressor
import pickle

def summarize_rule(rule_ls,show):
    string_rule = rule_ls
    group = []
    count = []
    loss_info = []
    
    for i in range(len(string_rule)):
        if_inside = string_rule[i] in group
        if not if_inside:
            group.append(string_rule[i])
            count.append(1)
        else:
            idx = group.index(string_rule[i])
            count[idx] += 1
    if show==True:
        for i in range(len(group)):
            print(group[i],"\t",round(count[i]/len(string_rule)*100),"%")
    else:
        return group, np.array(count)/len(string_rule)
        
def summarize_loss(rule_ls,loss_best_ls):
    string_rule = rule_ls
    group = []
    loss_info = []

    for i in range(len(string_rule)):
        if_inside = string_rule[i] in group
        if not if_inside:
            group.append(string_rule[i])
            loss_info.append(loss_best_ls[i])
        else:
            idx = group.index(string_rule[i])
            loss_info[idx] = np.min([loss_info[idx],loss_best_ls[i]])
    for i in range(len(group)):
        print(group[i],"\t",loss_info[i])
            
def get_result_sb(string,show=True):
    file = string
    with open(file+'.pkl', 'rb') as f:
        result = pickle.load(f)
    rule_nonstab = result[0]
    rule_stab = result[1]
    n_ls = result[2]
    group_ls = result[3]
    rule_fit0_ls = result[4]
    rule_fit_ls = result[5]
    loss_best_ls = result[6]
    
    if show == True:
#         print(len(rule_nonstab))
        print("non stablize:")
        summarize_rule(rule_nonstab,show)
        print("\n")
        print("stablize:")
        summarize_rule(rule_stab,show)
#         print(n_ls)

#         print("\n")
#         print("test loss info:")
#         summarize_loss(rule_stab,loss_best_ls)
        
    else:
#         print(string)
#         print(len(rule_nonstab))
        group1, percent1 = summarize_rule(rule_nonstab,show)
        group2, percent2 = summarize_rule(rule_stab,show)
        
        return percent1, percent2
    
def extract_data(string_ls,feature,file):
    percent1_ls = []
    percent2_ls = []
    for string in string_ls:
        percent1, percent2 = get_result_sb(string,False)
        percent1_ls.append(np.sort(percent1)[::-1])
        percent2_ls.append(np.sort(percent2)[::-1])

    df = pd.DataFrame(data={'feature': [], 'rule': [], 'percent':[],'type':[]})

    for k in range(len(feature)):
        percent1 = percent1_ls[k]
        for i in range(len(percent1)):
            df.loc[len(df.index)] = [feature[k],i,percent1[i],"Non-Stab"]

        percent2 = percent2_ls[k]
        for i in range(len(percent2)):
            df.loc[len(df.index)] = [feature[k],i,percent2[i],"Stab"]

    df.to_csv(file,index=False)

In [5]:
get_result_sb("symbolic_data1_n_sim_10000_nmax_100000_len_max_3")

non stablize:
X11 + X3 + X4 + 1 	 66 %
X1 + X11 + X3 + X4 + 1 	 7 %
X1 + X3 + X4 	 16 %
X3 + X4 + X8 + 1 	 5 %
X1 + X11 + X3 + X4 + X8 	 3 %
X1 + X3 + X4 + X8 	 1 %
X1 + X3 + X4 + X8 + 1 	 1 %
X1 + X11 + X3 + X4 	 1 %


stablize:
X11 + X3 + X4 + 1 	 83 %
X1 + X3 + X4 	 10 %
X1 + X11 + X3 + X4 + X8 	 4 %
X1 + X11 + X3 + X4 + 1 	 1 %
X1 + X11 + X3 + X4 	 1 %
X1 + X3 + X4 + X8 	 1 %


In [6]:
# n_sim
string_ls = ["symbolic_data1_n_sim_1000_nmax_100000_len_max_3",
            "symbolic_data1_n_sim_2500_nmax_100000_len_max_3",
            "symbolic_data1_n_sim_5000_nmax_100000_len_max_3",
            "symbolic_data1_n_sim_7500_nmax_100000_len_max_3",
            "symbolic_data1_n_sim_10000_nmax_100000_len_max_3"]
feature = [1000,2500,5000,7500,10000]
file = "symbolic_data1_n_sim.csv"
extract_data(string_ls,feature,file)

# len_max
string_ls = ["symbolic_data1_n_sim_10000_nmax_100000_len_max_3",
            "symbolic_data1_n_sim_10000_nmax_100000_len_max_4",
            "symbolic_data1_n_sim_10000_nmax_100000_len_max_5",
            "symbolic_data1_n_sim_10000_nmax_100000_len_max_6",
            "symbolic_data1_n_sim_10000_nmax_100000_len_max_7"]
feature = [3,4,5,6,7]
file = "symbolic_data1_len_max.csv"
extract_data(string_ls,feature,file)

# nmax
string_ls = ["symbolic_data1_n_sim_10000_nmax_10000_len_max_3",
            "symbolic_data1_n_sim_10000_nmax_25000_len_max_3",
            "symbolic_data1_n_sim_10000_nmax_50000_len_max_3",
            "symbolic_data1_n_sim_10000_nmax_100000_len_max_3",
            "symbolic_data1_n_sim_10000_nmax_250000_len_max_3"]
feature = [10000,25000,50000,100000,250000]
file = "symbolic_data1_nmax.csv"
extract_data(string_ls,feature,file)

In [7]:
get_result_sb("symbolic_data2_n_sim_10000_nmax_100000_len_max_3")

non stablize:
X20 + X22 + X27 + X7 	 6 %
X13 + X20 + X22 + X27 	 1 %
X20 + X21 + X22 + X27 	 16 %
X20 + X22 + X23 + X27 	 48 %
X20 + X22 + X26 + X27 	 1 %
X20 + X23 + X27 + X7 	 2 %
X20 + X21 + X22 + X23 	 2 %
X13 + X20 + X23 + X27 	 2 %
X20 + X22 + X27 + X4 	 1 %
X20 + X22 + X23 + X24 	 1 %
X20 + X23 + X27 + 1 	 1 %
X20 + X22 + X23 + X7 	 4 %
X21 + X22 + X23 + X27 	 3 %
X1 + X20 + X22 + X26 	 1 %
X20 + X22 + X23 + 1 	 1 %
X12 + X20 + X22 + X27 	 1 %
X20 + X22 + X27 + X3 	 1 %
X20 + X22 + X27 	 4 %
X1 + X20 + X22 + X27 	 3 %
X20 + X22 + X26 + X7 	 1 %


stablize:
X20 + X22 + X23 + X27 	 95 %
X20 + X22 + X27 + X7 	 1 %
X1 + X20 + X22 + X27 	 1 %
X20 + X21 + X22 + X27 	 2 %
X20 + X22 + X27 	 1 %


In [8]:
# n_sim
string_ls = ["symbolic_data2_n_sim_1000_nmax_100000_len_max_3",
            "symbolic_data2_n_sim_2500_nmax_100000_len_max_3",
            "symbolic_data2_n_sim_5000_nmax_100000_len_max_3",
            "symbolic_data2_n_sim_7500_nmax_100000_len_max_3",
            "symbolic_data2_n_sim_10000_nmax_100000_len_max_3"]
feature = [1000,2500,5000,7500,10000]
file = "symbolic_data2_n_sim.csv"
extract_data(string_ls,feature,file)

# len_max
string_ls = ["symbolic_data2_n_sim_10000_nmax_100000_len_max_3",
            "symbolic_data2_n_sim_10000_nmax_100000_len_max_4",
            "symbolic_data2_n_sim_10000_nmax_100000_len_max_5",
            "symbolic_data2_n_sim_10000_nmax_100000_len_max_6",
            "symbolic_data2_n_sim_10000_nmax_100000_len_max_7"]
feature = [3,4,5,6,7]
file = "symbolic_data2_len_max.csv"
extract_data(string_ls,feature,file)

# nmax
string_ls = ["symbolic_data2_n_sim_10000_nmax_10000_len_max_3",
            "symbolic_data2_n_sim_10000_nmax_25000_len_max_3",
            "symbolic_data2_n_sim_10000_nmax_50000_len_max_3",
            "symbolic_data2_n_sim_10000_nmax_100000_len_max_3",
            "symbolic_data2_n_sim_10000_nmax_250000_len_max_3"]
feature = [10000,25000,50000,100000,250000]
file = "symbolic_data2_nmax.csv"
extract_data(string_ls,feature,file)